In [4]:
import math
import time


In [5]:
Xn = time.time()
Xn_2 = time.time()*1000
u_1 = 15
u_2 = 30
u_31 = 25
u_32 = 10
o_1 = o_2 = o_31 = o_32 = 5
        
def cong_mixto():
    global Xn
    a = 25214903917
    m = 2**48 - 1
    Xn = (a*Xn + 1) % m
    return Xn/m

In [6]:
def exponencial(media):
    U = cong_mixto()
    X = (math.log(U)/media)*-1
    return X

def normal_gen_scheimen(u, o):
    R = cong_mixto()
    Z = (R**0.135 - (1-R)**0.135)/0.1975
    X = u + Z*o
    if X < 0:
        X = 0
    return X

def uniforme(a, b):
    U = cong_mixto()
    X = (a + (b-a)*U)
    return X


In [7]:
def servicio_nodo_1(t_suceso):
    global t, T_LL1, T_S1, T_S2, T_S3, NS_1, NS_2, NS_3, N_LL1, N_LL2, N_LL3, n_1, n_2, n_3, n_med_n1, n_med_n2, n_med_n3, U, S, W, Z, l, LL_1, LL_2, LL_3, S_1, S_2, S_3
    n_med_n1 = n_med_n1 + n_1*(t_suceso - t)
    n_1 = n_1 - 1
    NS_1 = NS_1 + 1
    S_1.append(t_suceso)
    
    #Genero una uniforme para determinar a qué nodo me dirijo
    U = uniforme(0, 1)
    if U < 0.4:
        n_med_n2 = n_med_n2 + n_2*(t_suceso - t)
        n_2 = n_2 + 1
        N_LL2 = N_LL2 + 1
        LL_2.append(t_suceso)
        if n_2 == 1:
            Z = exponencial(u_2)
            T_S2 = t_suceso + Z
    else:
        n_med_n3 = n_med_n3 + n_3*(t_suceso - t)
        n_3 = n_3 + 1
        N_LL3 = N_LL3 + 1
        LL_3.append(t_suceso)
        if n_3 == 1:
            W = normal_gen_scheimen(u_31, o_31)
            T_S3 = t_suceso + W
    
    t = t_suceso
    if n_1 > 0:
        S = normal_gen_scheimen(u_1, o_1)
        T_S1 = t + S


In [8]:
def servicio_nodo_2(t_suceso):
    global t, T_S2, T_S3, NS_2, N_LL3, n_2, n_med_n2, n_3, n_med_n3, W, Y, u_2, u_31, o_31, LL_3, S_2
    n_med_n2 = n_med_n2 + n_2*(t_suceso - t)
    n_2 = n_2 - 1
    NS_2 = NS_2 + 1
    S_2.append(t_suceso)
    if n_2 > 0:
        Y = exponencial(u_2)
        T_S2 = t_suceso + Y
    
    n_med_n3 = n_med_n3 + n_3*(t_suceso - t)
    n_3 = n_3 + 1
    N_LL3 = N_LL3 + 1
    LL_3.append(t_suceso)
    if n_3 == 1:
        W = normal_gen_scheimen(u_31, o_31)
        T_S3 = t_suceso + W
    
    t = t_suceso
    

In [9]:
def servicio_nodo_3(t_suceso):
    global t, n_3, n_med_n3, NS_3, T_S3, S_3, R, u_31, u_32, o_31, o_32
    n_med_n3 = n_med_n3 + n_3*(t_suceso - t)
    n_3 = n_3 - 1
    NS_3 = NS_3 + 1
    S_3.append(t_suceso)
    if n_3 > 0:
        if n_3 < 5:
            R = normal_gen_scheimen(u_31, o_31)
        else:
            R = normal_gen_scheimen(u_32, o_32)
        T_S3 = t_suceso + R
    
    t = t_suceso


In [10]:
def llegada_cliente(t_suceso):
    global t, T, T_LL1, T_S1, NS_1, N_LL1, n_1, n_med_n1, Y, l, u_1, o_1, LL_1
    n_med_n1 = n_med_n1 + n_1*(t_suceso - t)
    n_1 = n_1 + 1
    N_LL1 = N_LL1 + 1
    LL_1.append(t_suceso)
    t = t_suceso
    
    Y = exponencial(l)
    if t+Y < T:
        T_LL1 = t + Y
    if n_1 == 1:
        Y = normal_gen_scheimen(u_1, o_1)
        T_S1 = t + Y
        



In [12]:

T = 1e7

t = t_suc = Tp = 0
T_LL1 = T_S1 = T_S2 = T_S3 = math.inf
N_LL1 = N_LL2 = N_LL3 = NS_1 = NS_2 = NS_3 = n_1 = n_2 = n_3 = 0
LL_1 = []
LL_2 = []
LL_3 = []
S_1 = []
S_2 = []
S_3 = []
n_med_n1 = n_med_n2 = n_med_n3 = 0
l = 1/40

X = exponencial(l)
if X > T:
    Tp = t_medio_sistema = 0
    n_med_n1 = n_med_n2 = n_med_n3 = 0
    exit(0)
else:
    llegada_cliente(X)
    while T_LL1 != math.inf or T_S1 != math.inf or T_S2 != math.inf or T_S3 != math.inf :
        if min(T_LL1, T_S1, T_S2, T_S3) == T_LL1:
            t_suc = T_LL1
            T_LL1 = math.inf
            llegada_cliente(t_suc)
        if min(T_LL1, T_S1, T_S2, T_S3) == T_S1:
            t_suc = T_S1
            T_S1 = math.inf
            servicio_nodo_1(t_suc)
        if min(T_LL1, T_S1, T_S2, T_S3) == T_S2:
            t_suc = T_S2
            T_S2 = math.inf
            servicio_nodo_2(t_suc)
        if min(T_LL1, T_S1, T_S2, T_S3) == T_S3:
            t_suc = T_S3
            T_S3 = math.inf
            servicio_nodo_3(t_suc)
        
Tp = max(0, t-T)
cum1 = cum2 = cum3 = 0

ind = 0
while(ind < N_LL1):
    cum1 = cum1 + S_1[ind] - LL_1[ind]
    ind = ind + 1

ind = 0
while ind < N_LL2 :
    cum2 = cum2 + S_2[ind] - LL_2[ind]
    ind = ind + 1
    
ind = 0
while ind < N_LL3 :
    cum3 = cum3 + S_3[ind] - LL_3[ind]
    ind = ind + 1

t_medio_sistema = (cum1/N_LL1) + (0.4*(cum2/N_LL2)) + (cum3/N_LL3)
n_med_n1 = n_med_n1 / t
n_med_n2 = n_med_n2 / t
n_med_n3 = n_med_n3 / t
print("Clientes atendidos:", (NS_1 + NS_2 +NS_3))
print("Atendidos en servidor\n1 = {}\n2 = {}\n3 = {}".format(NS_1, NS_2, NS_3))
print("Tiempo medio en el sistema:", t_medio_sistema)


Clientes atendidos: 599496
Atendidos en servidor
1 = 249687
2 = 100122
3 = 249687
Tiempo medio en el sistema: 60.93693780286422
